In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('gdrive/My Drive/Grokking/reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
      
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))


word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)


In [0]:
print(len(raw_reviews))

25000


In [0]:
print("tokens len", len(tokens),tokens[0])

tokens len 25000 ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', '.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', '', 'such', 'as', '', 'teachers', '', '.', 'my', '', '', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'high', '', 's', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '', 'teachers', '', '.', 'the', 'scramble', 'to', 'survive', 'financially', '', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', '', 'pomp', '', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', '', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', '.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', '', 'i', 'immediately', 'recalled', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'at', '.', '.', 

In [0]:
print("len(vocab)", len(vocab),"\n",vocab[:10])

len(vocab) 74075 
 ['', 'inclusion', 'nosebleeds', 'cavil', 'horrizon', 'ribisi', 'socialized', 'fia', 'hefty', 'tax']


In [0]:
#print(word2index)

In [0]:
print(len(input_dataset),input_dataset[0:2])

25000 [[0, 0, 18279, 34940, 0, 51183, 73612, 22934, 54516, 13100, 73486, 39097, 15345, 0, 37918, 39097, 19019, 30727, 15137, 72817, 0, 70594, 47141, 39097, 22186, 42934, 57381, 8496, 50239, 45935, 7787, 70594, 25766, 23626, 50239, 31118, 73612, 54476, 24975, 32528, 39107, 59293, 60564, 0, 30803, 53723, 57381, 22934, 51183, 7016, 49794, 73612, 46940, 56160, 0, 66565, 57381, 28674, 51183, 58776, 47966, 28357, 58776, 57381, 23164, 50239, 37263, 49794, 57381, 14562, 42934, 41029, 45935, 1455, 32648, 49794, 25766, 24677, 0, 8598, 27014, 42934, 30727, 57381, 33686, 50239, 39318, 62476, 0, 0, 73612, 13138, 39720, 0, 0, 16533, 50239, 25766, 24677, 0, 8598, 36530, 0, 70594, 25766, 53103, 0, 45935, 30342, 1055, 47524, 25766, 23626, 73612, 63152, 42934, 0, 18233, 0, 0, 0, 18233, 0, 0, 1869, 58776, 49794, 30143, 5685, 1055, 69356, 45420, 0, 54496, 58209, 36998, 68446, 50717, 57381, 68199, 42934, 17422, 11072, 73970, 34319, 20051, 16426, 57381, 37263, 67385, 0, 66565, 24594, 33739, 57381, 65325, 45

In [7]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2  #shape: (74075, 50)
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0              #shape: (74075, 50)

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):
    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])   

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]
    
    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])       #layer_1_delta.shape (50,)
    
    weights_0_1[left_context+right_context] -= layer_1_delta * alpha     #shape: (3, 50)
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha #shape: (6, 50)

  if(rev_i % 250 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
  sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))
print(similar('terrible'))

Progress:0.99998[('terrible', -0.0), ('horrible', -2.9619505315492067), ('brilliant', -3.448479630886247), ('mediocre', -3.7561402059622595), ('superb', -3.764602798538214), ('phenomenal', -3.8376770826096203), ('pathetic', -3.8474707652398283), ('great', -3.8954507232213773), ('dreadful', -4.062210317422653), ('remarkable', -4.0783091929144994)]
